In [2]:
import geopandas as gpd
from owslib.wfs import WebFeatureService
from requests import Request

In [18]:
!pwd

/home/holmes/plandata_delta


In [23]:
wfs = WebFeatureService(url='https://geoserver.plandata.dk/geoserver/wfs', version='2.0.0')
base_request_str = 'https://geoserver.plandata.dk/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeNames='

# Create an empty GeoPackage file or use an existing one
output_gpkg = './data/plandata.gpkg'

for layer in wfs.contents:
    print(layer)
    url = base_request_str + layer

    q = Request('GET', url).prepare().url
    print(q)
    layer_file_bame =layer.replace(':', '_')
    df = gpd.read_file(q, format='GML')
    df.crs = 'EPSG:25832'


lpd:lpd_bp_Stationer
https://geoserver.plandata.dk/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeNames=lpd:lpd_bp_Stationer
lpd:lpd_bp_arealreservation
https://geoserver.plandata.dk/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeNames=lpd:lpd_bp_arealreservation


UnsupportedGeometryTypeError: 12

https://geoserver.plandata.dk/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeNames=lpd:lpd_bp_arealreservation&maxfeatures=1000

In [25]:
wfs = WebFeatureService(url='https://geoserver.plandata.dk/geoserver/wfs', version='2.0.0')
base_request_str = 'https://geoserver.plandata.dk/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeNames='



for layer in wfs.contents:
    print(layer)
    url = base_request_str + layer + '&maxfeatures=1000'
    # Make the request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Save the response content as an XML file
        layer_file_bame =layer.replace(':', '_')
        with open(layer_file_bame+'.gml', 'wb') as file:
            file.write(response.content)
    else:
        print(f"Request failed with status code: {response.status_code}")


lpd:lpd_bp_Stationer
lpd:lpd_bp_arealreservation
lpd:lpd_bp_byggezoner_stationer
lpd:lpd_bp_kompressorstation_everdrup
lpd:lpd_bp_modtageterminal_nybro
lpd:lpd_bp_oplag_og_arbplads
lpd:lpd_gaslf_arealreservation
lpd:lpd_gaslf_midlertidige_arbejdspladser
lpd:lpd_gaslf_stationer
lpd:lpd_gaslf_stationsareal
lpd:lpd_hoevsoere_maaleomraade_region
lpd:lpd_oesterild_maaleomraade_region
lpd:lpd_omplacering_sommerhusomr_vedt_2019_skal_tilbagefoeres
lpd:lpd_omplacering_sommerhusomr_vedt_2019_tilbagefoerelse
lpd:lpd_omplacering_sommerhusomr_vedt_2019_udlaeg
lpd:lpd_omplacering_sommerhusomr_vedt_2019_udlaeg_betinget
lpd:lpd_sommerhus_tilbagefoerelse_20190315
lpd:lpd_sommerhus_udlaeg_20190315
lpd:lpd_udviklingsomr_vedt_2019
lpd:lpd_udviklingsomr_vedt_2019_paragraf3
lpd:lpd_udviklingsomr_vedt_2019_paragraf5
lpd:lpd_udviklingsomr_vedt_2019_traadt_i_kraft
lpd:lpd_udviklingsomr_vedt_2021
lpd:lpd_udviklingsomr_vedt_2021_paragraf3
lpd:lpd_udviklingsomr_vedt_2021_paragraf5
lpd:lpd_udviklingsomr_vedt_2021_

KeyboardInterrupt: 

In [19]:
wfs = WebFeatureService(url='https://geoserver.plandata.dk/geoserver/wfs', version='2.0.0')
base_request_str = 'https://geoserver.plandata.dk/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeNames='

# Create an empty GeoPackage file or use an existing one
output_gpkg = './data/plandata.gpkg'

for layer in wfs.contents:
    print(layer)
    url = base_request_str + layer

    q = Request('GET', url).prepare().url
    try:
        df = gpd.read_file(q, format='GML')
        df.crs = 'EPSG:25832'
        layer_file_bame =layer.replace(':', '_')
        #df.to_file(layer_file_bame+'.shp')
        gdf.to_file(output_gpkg, layer=layer, driver="GPKG", mode='a' if os.path.exists(output_gpkg) else 'w')
    except Exception as e:
        print(f"An error occurred: {e}")
        continue  # This is optional as the loop will naturally go to the next iteration
    



lpd:lpd_bp_Stationer
lpd:lpd_bp_arealreservation
An error occurred: 12
lpd:lpd_bp_byggezoner_stationer
An error occurred: 12
lpd:lpd_bp_kompressorstation_everdrup
An error occurred: 12
lpd:lpd_bp_modtageterminal_nybro
An error occurred: 12
lpd:lpd_bp_oplag_og_arbplads
An error occurred: 12
lpd:lpd_gaslf_arealreservation
An error occurred: 12
lpd:lpd_gaslf_midlertidige_arbejdspladser
An error occurred: 12
lpd:lpd_gaslf_stationer
An error occurred: 12
lpd:lpd_gaslf_stationsareal
An error occurred: 12
lpd:lpd_hoevsoere_maaleomraade_region
An error occurred: 1012
lpd:lpd_oesterild_maaleomraade_region
An error occurred: 1012
lpd:lpd_omplacering_sommerhusomr_vedt_2019_skal_tilbagefoeres
An error occurred: 12
lpd:lpd_omplacering_sommerhusomr_vedt_2019_tilbagefoerelse
An error occurred: 12
lpd:lpd_omplacering_sommerhusomr_vedt_2019_udlaeg
An error occurred: 12
lpd:lpd_omplacering_sommerhusomr_vedt_2019_udlaeg_betinget
An error occurred: 12
lpd:lpd_sommerhus_tilbagefoerelse_20190315
An error oc

KeyboardInterrupt: 

this can be expanded to 

    First request that starts from the beginning

http://wms.pcn.minambiente.it/ogc?map=/ms_ogc/wfs/Edifici.map&service=WFS&version=1.1.0&request=GetFeature&typename=ED.EDIFICATO.CAPOLUOGHI.&maxfeatures=1000

Save response into file

    Second request

http://wms.pcn.minambiente.it/ogc?map=/ms_ogc/wfs/Edifici.map&service=WFS&version=1.1.0&request=GetFeature&typename=ED.EDIFICATO.CAPOLUOGHI.&MaxFeatures=1000&StartIndex=1000

In [4]:
def download_wfs_layers_to_geopackage(url, output_filepath):
    # Connect to the WFS service
    wfs = WebFeatureService(url=url)
    
    # Check WFS version and set typename parameter accordingly
    typename_param = 'typename'
    if wfs.version == '2.0.0':
        typename_param = 'typenames'

    # For each layer, download its data
    for layer in wfs.contents:
        print(layer)
        # Check if layer name contains namespace (like 'namespace:layername')
        if ':' not in layer:
            layer = f"{wfs.provider.name}:{layer}"
        
        params = {
            typename_param: layer,
            'outputFormat': 'application/json'
        }
        
        try:
            response = wfs.getfeature(**params)
            data = response.read()
            gdf = gpd.read_file(data)
            
            # Append the data to the GeoPackage
            mode = 'a' if 'tables' in gpd.io.file.fiona.drvsupport.supported_drivers['GPKG'] and layer in gpd.io.file.fiona.drvsupport.supported_drivers['GPKG']['tables'] else 'w'
            gdf.to_file(output_filepath, layer=layer, driver="GPKG", mode=mode)
            print(f"Saved {layer} to {output_filepath}")
        except Exception as e:
            print(f"Error fetching data for layer {layer}: {e}")


if __name__ == '__main__':
    wfs_url = 'https://geoserver.plandata.dk/geoserver/wfs?servicename=wfs&request=getcapabilities&service=wfs'
    output_filepath = './data/plandata.gpkg'  # e.g., './data.gpkg'
    download_wfs_layers_to_geopackage(wfs_url, output_filepath)


lpd:lpd_bp_Stationer
Error fetching data for layer lpd:lpd_bp_Stationer: Feature type :l unknown
lpd:lpd_bp_arealreservation
Error fetching data for layer lpd:lpd_bp_arealreservation: Feature type :l unknown
lpd:lpd_bp_byggezoner_stationer
Error fetching data for layer lpd:lpd_bp_byggezoner_stationer: Feature type :l unknown
lpd:lpd_bp_kompressorstation_everdrup
Error fetching data for layer lpd:lpd_bp_kompressorstation_everdrup: Feature type :l unknown
lpd:lpd_bp_modtageterminal_nybro
Error fetching data for layer lpd:lpd_bp_modtageterminal_nybro: Feature type :l unknown
lpd:lpd_bp_oplag_og_arbplads
Error fetching data for layer lpd:lpd_bp_oplag_og_arbplads: Feature type :l unknown
lpd:lpd_gaslf_arealreservation
Error fetching data for layer lpd:lpd_gaslf_arealreservation: Feature type :l unknown
lpd:lpd_gaslf_midlertidige_arbejdspladser
Error fetching data for layer lpd:lpd_gaslf_midlertidige_arbejdspladser: Feature type :l unknown
lpd:lpd_gaslf_stationer
Error fetching data for laye

In [5]:
from owslib.wfs import WebFeatureService
wfs = WebFeatureService(url='https://geoserver.plandata.dk/geoserver/wfs', version='2.0.0')


'Plandata WFS Service'

In [6]:
[operation.name for operation in wfs11.operations]

['GetCapabilities',
 'DescribeFeatureType',
 'GetFeature',
 'GetPropertyValue',
 'ListStoredQueries',
 'DescribeStoredQueries',
 'CreateStoredQuery',
 'DropStoredQuery',
 'ImplementsBasicWFS',
 'ImplementsTransactionalWFS',
 'ImplementsLockingWFS',
 'KVPEncoding',
 'XMLEncoding',
 'SOAPEncoding',
 'ImplementsInheritance',
 'ImplementsRemoteResolve',
 'ImplementsResultPaging',
 'ImplementsStandardJoins',
 'ImplementsSpatialJoins',
 'ImplementsTemporalJoins',
 'ImplementsFeatureVersioning',
 'ManageStoredQueries',
 'PagingIsTransactionSafe',
 'QueryExpressions']

In [7]:
list(wfs11.contents)

['lpd:lpd_bp_Stationer',
 'lpd:lpd_bp_arealreservation',
 'lpd:lpd_bp_byggezoner_stationer',
 'lpd:lpd_bp_kompressorstation_everdrup',
 'lpd:lpd_bp_modtageterminal_nybro',
 'lpd:lpd_bp_oplag_og_arbplads',
 'lpd:lpd_gaslf_arealreservation',
 'lpd:lpd_gaslf_midlertidige_arbejdspladser',
 'lpd:lpd_gaslf_stationer',
 'lpd:lpd_gaslf_stationsareal',
 'lpd:lpd_hoevsoere_maaleomraade_region',
 'lpd:lpd_oesterild_maaleomraade_region',
 'lpd:lpd_omplacering_sommerhusomr_vedt_2019_skal_tilbagefoeres',
 'lpd:lpd_omplacering_sommerhusomr_vedt_2019_tilbagefoerelse',
 'lpd:lpd_omplacering_sommerhusomr_vedt_2019_udlaeg',
 'lpd:lpd_omplacering_sommerhusomr_vedt_2019_udlaeg_betinget',
 'lpd:lpd_sommerhus_tilbagefoerelse_20190315',
 'lpd:lpd_sommerhus_udlaeg_20190315',
 'lpd:lpd_udviklingsomr_vedt_2019',
 'lpd:lpd_udviklingsomr_vedt_2019_paragraf3',
 'lpd:lpd_udviklingsomr_vedt_2019_paragraf5',
 'lpd:lpd_udviklingsomr_vedt_2019_traadt_i_kraft',
 'lpd:lpd_udviklingsomr_vedt_2021',
 'lpd:lpd_udviklingsomr_

In [14]:
from requests import Request
#url = 'http://wms.pcn.minambiente.it/ogc?map=/ms_ogc/wfs/Edifici.map&service=wfs&request=getFeature&version=1.0.0&typename=ED.EDIFICATO.CAPOLUOGHI.'
url = 'https://geoserver.plandata.dk/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeNames=lpd:lpd_bp_Stationer'


q = Request('GET', url).prepare().url
df = gpd.read_file(q, format='GML')
df.crs = 'EPSG:25832'

df.to_file('output.shp')

In [15]:
import geopandas as gpd
from owslib.wfs import WebFeatureService
import os

# Define the WFS endpoint
url = 'https://geoserver.plandata.dk/geoserver/wfs'

# Connect to the WFS service
wfs = WebFeatureService(url)

# List feature types (layers) available in the service
layers = list(wfs.contents.keys())

# Create an empty GeoPackage file or use an existing one
output_gpkg = 'output.gpkg'

for layer in layers:
    # Build the request URL for each layer
    print(layer)
    layer_url = f"{url}?service=WFS&version=2.0.0&request=GetFeature&typeNames={layer}"
    # Read the layer into a GeoDataFrame
    gdf = gpd.read_file(layer_url)
    
    # Set the Coordinate Reference System (CRS)
    gdf.crs = 'EPSG:25832'
    
    # Save the GeoDataFrame to the GeoPackage
    gdf.to_file(output_gpkg, layer=layer, driver="GPKG", mode='a' if os.path.exists(output_gpkg) else 'w')

print(f"All layers saved to {output_gpkg}")

lpd:lpd_bp_Stationer
lpd:lpd_bp_arealreservation


UnsupportedGeometryTypeError: 12

In [9]:
[storedquery.id for storedquery in wfs11.storedqueries]

['urn:ogc:def:query:OGC-WFS::GetFeatureById']

In [12]:
[parameter.name for parameter in wfs11.storedqueries[0].parameters]

['ID']

In [ ]:
response = wfs11.getfeature(storedQueryID='urn:ogc:def:query:OGC-WFS::GetFeatureById', storedQueryParams={'ID':'gmd_ex.1'})

[operation.name for operation in wfs11.operations]